In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import pandas as pd
import os
import random
import warnings
warnings.filterwarnings('ignore')
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import numpy as np
from utils import custom_train_test_split, lgbm_predict, post_slack

base_feats = ['KnowledgeTag', 'user_correct_answer', 'user_total_answer', 
         'user_acc', 'test_mean', 'test_sum', 'tag_mean','tag_sum']
# train과 test 데이터셋은 사용자 별로 묶어서 분리를 해주어야함

## 1. 데이터 로딩
data_dir = '/opt/ml/input/data' # 경로
# train_file_path = os.path.join(data_dir, 'train_data.csv') # 데이터
# test_file_path = os.path.join(data_dir, 'test_data.csv')
after_fe_path = os.path.join(data_dir, 'after_fe_train_test.pkl')
df = pd.read_pickle(after_fe_path)

train_df = df[df.kind=='train']
train, valid = custom_train_test_split(train_df, ratio=0.7) # 훈련데이터 split
test = df[df.kind=='test'] # 테스트 데이터
train2 = test[test.answerCode!=-1] # 테스트데이터 마지막 제출 2번쨰꺼까지 훈련데이터로 사용
train = pd.concat([train_df,train2]) # 훈련데이터 병합
train.shape, valid.shape, test.shape

x_train = train.drop('answerCode',axis=1)
y_train = train[['answerCode']]

x_valid = valid.drop('answerCode',axis=1)
y_valid = valid[['answerCode']]
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

((2525956, 24), (2525956, 1), (2007, 24), (2007, 1))

In [2]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec'
        ]

params = {
#     "max_depth": 8,  # 8,
#     "min_data_in_leaf": 1000,
    # "feature_fraction": 0.6,  # 0.8,
#     "bagging_fraction": 0.75,
    # "max_cat_group": 64,
    "objective": "binary",
#     "boosting": "gbdt",  # dart
#     "learning_rate": 0.01,  # 0.01,
    # "bagging_freq": 5,
    "seed": 42,
    # "max_bin": 50,
#     "num_leaves": 80,  # 40,
#     "metric": "auc",
}

model_auc = lgb.LGBMClassifier(
    **params,
    n_estimators=3200,
    silent=-1,
)


## 문제푼 시간 카테고리 변수도 추가

In [11]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec',
         'time_category',
        ]

In [4]:
lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

model5 = lgb.train(
    params, 
    lgb_x_train,
    valid_sets=[lgb_x_valid],
    verbose_eval=100,
    num_boost_round=3200,
#     early_stopping_rounds=200,
)

preds = model5.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.067799 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3105
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
[100]	valid_0's binary_logloss: 0.58363
[200]	valid_0's binary_logloss: 0.576
[300]	valid_0's binary_logloss: 0.570863
[400]	valid_0's binary_logloss: 0.563467
[500]	valid_0's binary_logloss: 0.558677
[600]	valid_0's binary_logloss: 0.556323
[700]	valid_0's binary_logloss: 0.552457
[800]	valid_0's binary_logloss: 0.550542
[900]	valid_0's binary_logloss: 0.549609
[1000]	valid_0's binary_logloss: 0.546317
[1100]	valid_0's binary_loglo

In [46]:
lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

model5 = lgb.train(
    params, 
    lgb_x_train,
    valid_sets=[lgb_x_valid],
    verbose_eval=100,
    num_boost_round=3200,
#     early_stopping_rounds=200,
)

preds = model5.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.059999 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3105
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
[100]	valid_0's binary_logloss: 0.560899
[200]	valid_0's binary_logloss: 0.552517
[300]	valid_0's binary_logloss: 0.547221
[400]	valid_0's binary_logloss: 0.540192
[500]	valid_0's binary_logloss: 0.535174
[600]	valid_0's binary_logloss: 0.531541
[700]	valid_0's binary_logloss: 0.527031
[800]	valid_0's binary_logloss: 0.523995
[900]	valid_0's binary_logloss: 0.522249
[1000]	valid_0's binary_logloss: 0.519503
[1100]	valid_0's binary_l

In [5]:
post_slack("done")

In [6]:
lgbm_predict(test, model5, FEATS, 'LGBM_Baseline_문제푼시간_추가_최대3600으로_time_category추가.csv')

writing prediction : output/LGBM_Baseline_문제푼시간_추가_최대3600으로_time_category추가.csv


## 문제푼 시간 3600초 이상 모두 3600으로 변환

In [24]:
lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

model4 = lgb.train(
    params, 
    lgb_x_train,
    valid_sets=[lgb_x_valid],
    verbose_eval=100,
    num_boost_round=3200,
#     early_stopping_rounds=200,
)

preds = model4.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.229645 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3095
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
[100]	valid_0's binary_logloss: 0.564848
[200]	valid_0's binary_logloss: 0.556382
[300]	valid_0's binary_logloss: 0.55063
[400]	valid_0's binary_logloss: 0.543942
[500]	valid_0's binary_logloss: 0.5388
[600]	valid_0's binary_logloss: 0.534408
[700]	valid_0's binary_logloss: 0.52949
[800]	valid_0's binary_logloss: 0.527017
[900]	valid_0's binary_logloss: 0.525339
[1000]	valid_0's binary_logloss: 0.521399
[1100]	valid_0's binary_logloss: 0.51872
[1200]	valid_0's binary_logloss: 0.516894
[1300]	val

In [7]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec'
        ]

x_train.shape, x_valid.shape

lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

model4 = lgb.train(
    params, 
    lgb_x_train,
    valid_sets=[lgb_x_valid],
    verbose_eval=100,
    num_boost_round=3200,
#     early_stopping_rounds=200,
)

preds = model4.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

post_slack("done")
lgbm_predict(test, model4, FEATS, 'LGBM_Baseline_문제푼시간_추가_3600초이상모두3600으로.csv')

((2525956, 24), (2007, 24))

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.039673 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3095
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
[100]	valid_0's binary_logloss: 0.583624
[200]	valid_0's binary_logloss: 0.575627
[300]	valid_0's binary_logloss: 0.57101
[400]	valid_0's binary_logloss: 0.565064
[500]	valid_0's binary_logloss: 0.560446
[600]	valid_0's binary_logloss: 0.557313
[700]	valid_0's binary_logloss: 0.553471
[800]	valid_0's binary_logloss: 0.549933
[900]	valid_0's binary_logloss: 0.54842
[1000]	valid_0's binary_logloss: 0.54517
[1100]	valid_0's binary_logl

## 문제푼 시간 4800초 이상 모두 4800초로 변환

In [17]:
lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

model3 = lgb.train(
    params, 
    lgb_x_train,
    valid_sets=[lgb_x_valid],
    verbose_eval=100,
    num_boost_round=3200,
#     early_stopping_rounds=200,
)

preds = model3.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.063728 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3095
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
[100]	valid_0's binary_logloss: 0.570995
[200]	valid_0's binary_logloss: 0.563762
[300]	valid_0's binary_logloss: 0.558203
[400]	valid_0's binary_logloss: 0.551398
[500]	valid_0's binary_logloss: 0.546308
[600]	valid_0's binary_logloss: 0.542565
[700]	valid_0's binary_logloss: 0.5393
[800]	valid_0's binary_logloss: 0.535955
[900]	valid_0's binary_logloss: 0.533331
[1000]	valid_0's binary_logloss: 0.531025
[1100]	valid_0's binary_log

In [18]:
post_slack("done")

In [19]:
lgbm_predict(test, model2, FEATS, 'LGBM_Baseline_문제푼시간_추가_4800초로수렴.csv')

writing prediction : output/LGBM_Baseline_문제푼시간_추가_4800초로수렴.csv


## 문제푼 시간 초로만 추가한 것

In [5]:
lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

model2 = lgb.train(
    params, 
    lgb_x_train,
    valid_sets=[lgb_x_valid],
    verbose_eval=100,
    num_boost_round=3200,
#     early_stopping_rounds=200,
)

preds = model2.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.069061 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3095
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
[100]	valid_0's binary_logloss: 0.582419
[200]	valid_0's binary_logloss: 0.575168
[300]	valid_0's binary_logloss: 0.568893
[400]	valid_0's binary_logloss: 0.563229
[500]	valid_0's binary_logloss: 0.559829
[600]	valid_0's binary_logloss: 0.556637
[700]	valid_0's binary_logloss: 0.553144
[800]	valid_0's binary_logloss: 0.550667
[900]	valid_0's binary_logloss: 0.548314
[1000]	valid_0's binary_logloss: 0.546135
[1100]	valid_0's binary_l

In [6]:
post_slack("done")

In [5]:
model_auc.fit(
    X=x_train[FEATS],
    y=y_train,
    early_stopping_rounds=1000,
    eval_set=[(x_train[FEATS], y_train), (x_valid[FEATS], y_valid)],
    eval_names=["train", "valid"],
    eval_metric="auc",
    verbose=100,
)

preds = model_auc.predict_proba(x_valid[FEATS])[:, 1]
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)
print(f'VALID AUC : {auc} ACC : {acc}\n')

[100]	train's auc: 0.757435	train's binary_logloss: 0.546374	valid's auc: 0.70322	valid's binary_logloss: 0.62917
[200]	train's auc: 0.762059	train's binary_logloss: 0.54251	valid's auc: 0.710404	valid's binary_logloss: 0.623287
[300]	train's auc: 0.766217	train's binary_logloss: 0.539063	valid's auc: 0.714973	valid's binary_logloss: 0.619211
[400]	train's auc: 0.769646	train's binary_logloss: 0.536233	valid's auc: 0.721349	valid's binary_logloss: 0.614343
[500]	train's auc: 0.772549	train's binary_logloss: 0.533777	valid's auc: 0.724012	valid's binary_logloss: 0.611957
[600]	train's auc: 0.775027	train's binary_logloss: 0.531641	valid's auc: 0.726944	valid's binary_logloss: 0.609731
[700]	train's auc: 0.777064	train's binary_logloss: 0.52987	valid's auc: 0.729365	valid's binary_logloss: 0.607706
[800]	train's auc: 0.779319	train's binary_logloss: 0.527932	valid's auc: 0.731309	valid's binary_logloss: 0.606041
[900]	train's auc: 0.781924	train's binary_logloss: 0.525736	valid's auc: 0.

LGBMClassifier(n_estimators=3200, objective='binary', seed=42, silent=-1)

VALID AUC : 0.7722212283840191 ACC : 0.6925759840558047



In [8]:
lgbm_predict(test, model_auc, FEATS, 'LGBM_Baseline_학습데이터변경.csv')

writing prediction : output/LGBM_Baseline_학습데이터변경.csv


In [7]:
lgbm_predict(test, model2, FEATS, 'LGBM_Baseline_문제푼시간_추가.csv')

writing prediction : output/LGBM_Baseline_문제푼시간_추가.csv


In [21]:
d4 = pd.read_csv('output/LGBM_Baseline_학습데이터변경_logloss_기존7627.csv')